In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import matplotlib.pyplot as plt

from analysis.src.data import Data

In [ ]:
data_r = Data(model="resistor")
# data_r.num_datum, data_r.check_volts_ext(), data_r.check_num_cycle()

In [ ]:
data_r.compute_macro()

---
---
---

In [ ]:
data_rc = Data(model="resistor_capacitor")
# data_rc.num_datum, data_rc.check_volts_ext(), data_rc.check_num_cycle()

In [ ]:
data_rc.compute_macro()

---
---
---

In [ ]:
data_rcb = Data(model="resistor_capacitor_battery")
data_rcb.num_datum, data_rcb.check_volts_ext(), data_rcb.check_num_cycle()

In [ ]:
data_rcb.compute_macro()

In [ ]:
data_rcb.total_eb[:, :, data_rcb.idxs_width[(data_rcb.widths_unique == 0.25)]].shape

In [ ]:
data_rcb.idxs_width[(data_rcb.widths_unique == 0.25)]

---
---
---

In [ ]:
import matplotlib.ticker as tck

fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=72)

for width in data_r.widths_unique[-1:]:
    for length in data_r.lengths_unique:
        ax.scatter(np.log(length), data_r.macro_vext[length, width])

# === y축: log scale ===
ax.set_yscale('log')
ax.set_ylim(0.3, 0.8)

# ---- major ticks (네가 원했던 그대로) ----
y_major = np.arange(3, 9) / 10.0    # 0.3, 0.4, ..., 0.8
ax.set_yticks(y_major)
ax.set_yticklabels(y_major)

# ---- minor ticks (로그 간격, 레이블 없음) ----
y_minor = []
for y1, y2 in zip(y_major[:-1], y_major[1:]):
    # y1~y2 사이를 로그 간격으로 5등분 → 중간 3개를 minor로 사용
    y_minor.extend(np.geomspace(y1, y2, 5)[1:-1])
ax.set_yticks(y_minor, minor=True)
ax.yaxis.set_minor_formatter(tck.NullFormatter())

# === x축 ===
ax.set_xticks(np.linspace(2, 5, 7))         # major
ax.xaxis.set_minor_locator(tck.AutoMinorLocator())  # minor 자동 생성 (레이블 없음)

# === tick 스타일 ===
ax.tick_params(axis='both', direction='in', which='major',
               length=15, pad=7.5, labelsize=15)
ax.tick_params(axis='both', direction='in', which='minor',
               length=7.5)

plt.show()


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=72)

for width in data_r.widths_unique[3:]:
    for length in data_r.lengths_unique:
        # plt.scatter(length, data_r.macro_vext[length, width])
        ax.scatter(length, data_r.macro_nb[length, width])

ax.set_xticks(data_r.lengths_unique)
ax.set_xscale('log'); ax.set_yscale('log')

---
---
---

In [ ]:
total_vext = []
for data in [data_b001, data_b01, data_b1, data_b10, data_b100]:
    temp_length = []
    for length in data.lengths_unique:
        temp_width = []
        for width in data.widths_unique:
            temp_width.append(data.macro_vext[length, width])
        temp_length.append(temp_width)
    total_vext.append(temp_length)
total_vext = np.array(total_vext, dtype=np.float64) # (val_cap, length, width) -> <vext> (really vb - v_ the normalized quantity, not vb itself)

total_nb = []
for data in [data_b001, data_b01, data_b1, data_b10, data_b100]:
    temp_length = []
    for length in data.lengths_unique:
        temp_width = []
        for width in data.widths_unique:
            temp_width.append(data.macro_nb[length, width])
        temp_length.append(temp_width)
    total_nb.append(temp_length)
total_nb = np.array(total_nb, dtype=np.float64) # (val_cap, length, width) -> <nb> (really x the normalized quantity, not nb itself)

total_ecap = []
for data in [data_b001, data_b01, data_b1, data_b10, data_b100]:
    temp_length = []
    for length in data.lengths_unique:
        temp_width = []
        for width in data.widths_unique:
            temp_width.append(data.macro_ecap[length, width] / length )
        temp_length.append(temp_width)
    total_ecap.append(temp_length)
total_ecap = np.array(total_ecap, dtype=np.float64) # (val_cap, length, width) -> <ecap> (still not normalized yet; will do after like check unormalized and more study so its meaningful)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
import matplotlib as mpl

def plot_total_3d(total, data, val_caps, *,
                  normalize="none",           # "none", "L", "W", "LW"
                  cmap_name="jet_r",
                  elev=22, azim=-60,
                  s=18, lw=1.5,
                  metric_label=r"$\langle\mathrm{metric}\rangle$",
                  plane_color="0.92", plane_alpha=0.28):
    """
    total:     np.ndarray (n_val_cap, n_length, n_width)
    data:      Data instance (uses data.lengths_unique, data.widths_unique)
    val_caps:  list/1D array of capacitance values (len == total.shape[0])
    """

    lengths  = np.asarray(data.lengths_unique, dtype=float)
    widths   = np.asarray(data.widths_unique,  dtype=float)
    val_caps = np.asarray(val_caps, dtype=float)

    assert total.shape == (len(val_caps), len(lengths), len(widths)), "shape mismatch"

    # ln(c) on linear axis; label ticks as c
    y_vals = np.log(val_caps)

    # apply normalization for z-limits & plotting
    if normalize == "L":
        total_norm = total / lengths[None, :, None]
    elif normalize == "W":
        total_norm = total / widths[None, None, :]
    elif normalize == "LW":
        total_norm = total / (lengths[None, :, None] * widths[None, None, :])
    else:
        total_norm = total

    zmin = float(np.nanmin(total_norm))
    zmax = float(np.nanmax(total_norm))

    # colors by width
    nw = len(widths)
    colors = plt.colormaps[cmap_name](np.linspace(0.1, .9, nw))[:, :3]

    fig = plt.figure(figsize=(9, 7), dpi=150)
    ax = fig.add_subplot(111, projection='3d')

    # --- clean style: no background panes, no grid ---
    ax.grid(False)
    for axpane in (ax.xaxis.pane, ax.yaxis.pane, ax.zaxis.pane):
        axpane.set_edgecolor('none')
        axpane.set_facecolor((1, 1, 1, 0))  # transparent
        axpane.fill = False

    # --- faint planes at each capacitance (constant ln(c)) ---
    X = np.array([[lengths.min(), lengths.max()],
                  [lengths.min(), lengths.max()]], dtype=float)
    Z = np.array([[zmin, zmin],
                  [zmax, zmax]], dtype=float)
    for vc_log in y_vals:
        Y = np.full_like(X, vc_log, dtype=float)
        ax.plot_surface(X, Y, Z, color=plane_color, alpha=plane_alpha,
                        rstride=1, cstride=1, edgecolor='none', linewidth=0)

    # --- lines + scatters per width within each val_cap slice ---
    for wi, w in enumerate(widths):
        col = colors[wi]
        for vi, vc_log in enumerate(y_vals):
            x = lengths
            y = np.full_like(x, vc_log, dtype=float)
            z = total_norm[vi, :, wi]

            ax.plot(x, y, z, lw=lw, color=col, alpha=0.95)
            ax.scatter(x, y, z, s=s, color=col, alpha=0.95, edgecolors='none')

    # legend for widths
    handles = [mpl.lines.Line2D([0],[0], color=colors[i], lw=2, label=f"w={widths[i]:.2f}") for i in range(nw)]
    ax.legend(handles=handles, title="Width", loc="upper left", bbox_to_anchor=(1.02, 1.0))

    # labels
    ax.set_xlabel("Length L")
    ax.set_ylabel("Capacitance c")
    ax.set_zlabel(metric_label)

    # y-ticks: display original c while positioning at ln(c)
    ax.set_yticks(y_vals)
    ax.set_yticklabels([f"{c:g}" for c in val_caps])

    # view + layout
    ax.view_init(elev=elev, azim=azim)
    fig.subplots_adjust(right=0.82)  # room for legend
    plt.show()


In [ ]:
val_caps = [data_b001.val_cap, data_b01.val_cap, data_b1.val_cap, data_b10.val_cap, data_b100.val_cap]

# Ecap
# plot_total_3d(total_ecap, data_b1, val_caps, metric_label=r"$\langle E_{\mathrm{cap}}\rangle$")

# Vext (already vb - v_ in your array)
plot_total_3d(total_vext, data_b1, val_caps, metric_label=r"$\langle v_b\rangle - v\_$")

# Nb (already normalized: ⟨Nb⟩/L - 1)
# plot_total_3d(total_nb,   data_b1, val_caps, metric_label=r"$\langle N_b\rangle/L - 1$")
